In [ ]:
##################################################################################################################################
#                                                                                                                                #
#                             An Explainable AI Framework for Satellite Network Anomaly Detection                                #
#                                                                                                                                #
#                                                                                                                                #
#                            *******   Risk-Aware Capacity Advisor for Satellite Networks  *******                               #
#                                                                                                                                #
#                                                                                                                                #
#                                                    University of Hull                                                          #
#                                                MSc Artificial Intelligence                                                     #
#                                                                                                                                #
#                                                      Amadiz Sabino                                                             #
#                                                                                                                                #
##################################################################################################################################

In [9]:
# ----------------------------------------------
# Step 1 - Mount drive and import libraries
# ----------------------------------------------

import os
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import HistGradientBoostingClassifier

import shap
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

BASE_DIR = Path("/content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses")
CAP_DIR = BASE_DIR / "artifacts_capacity"
CAP_DIR.mkdir(exist_ok=True)

BASE_DIR, CAP_DIR


Mounted at /content/drive


(PosixPath('/content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses'),
 PosixPath('/content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/artifacts_capacity'))

In [15]:
comm_path = BASE_DIR / "ses_comm_features.csv"

if comm_path.exists():
    comm = pd.read_csv(comm_path, parse_dates=["time"])
    comm = comm.sort_values("time").set_index("time")
    # Use a 10-minute grid for capacity planning
    idx = comm.index.floor("10min").unique()
else:
    # Fallback: synthetic 2-month window every 10 minutes
    idx = pd.date_range("2021-10-01", "2021-12-01", freq="10min", tz="UTC")

idx = idx.sort_values()
print("Time points:", len(idx), "from", idx[0], "to", idx[-1])

# Create a small synthetic beam set (you can rename these later)
beams = ["AMERICAS", "EUROPE", "AFRICA", "ASIA"]
beam_df = (
    pd.MultiIndex.from_product([idx, beams], names=["time", "beam_id"])
    .to_frame(index=False)
)

beam_df.head()

Time points: 6292 from 2021-10-18 07:20:00+00:00 to 2021-11-30 23:50:00+00:00


,time,beam_id
0,2021-10-18 07:20:00+00:00,AMERICAS
1,2021-10-18 07:20:00+00:00,EUROPE
2,2021-10-18 07:20:00+00:00,AFRICA
3,2021-10-18 07:20:00+00:00,ASIA
4,2021-10-18 07:30:00+00:00,AMERICAS


In [16]:
rng = np.random.default_rng(42)

# Basic capacity assumption (per beam) in Mbps
beam_capacity = {
    "AMERICAS": 1000,
    "EUROPE": 900,
    "AFRICA": 700,
    "ASIA": 1100,
}

def make_demand(row):
    t = row["time"]
    beam = row["beam_id"]

    # hour-of-day pattern (busy evening)
    hod = t.hour + t.minute / 60
    diurnal = 0.4 + 0.4 * np.sin((hod - 18) / 24 * 2 * np.pi)

    # weekday vs weekend
    # Monday=0 ... Sunday=6
    wd = t.weekday()
    weekend_factor = 0.8 if wd >= 5 else 1.0

    # beam bias
    beam_bias = {
        "AMERICAS": 1.0,
        "EUROPE": 0.9,
        "AFRICA": 0.7,
        "ASIA": 1.1,
    }[beam]

    noise = rng.normal(0, 0.05)
    demand_norm = np.clip(diurnal * weekend_factor * beam_bias + noise, 0.05, 1.2)
    return demand_norm

beam_df["demand_norm"] = beam_df.apply(make_demand, axis=1)
beam_df["capacity_mbps"] = beam_df["beam_id"].map(beam_capacity)
beam_df["demand_mbps"] = (beam_df["demand_norm"] * beam_df["capacity_mbps"]).clip(
    upper=beam_df["capacity_mbps"]
)

beam_df.head()


,time,beam_id,demand_norm,capacity_mbps,demand_mbps
0,2021-10-18 07:20:00+00:00,AMERICAS,0.278428,1000,278.427797
1,2021-10-18 07:20:00+00:00,EUROPE,0.184874,900,166.386189
2,2021-10-18 07:20:00+00:00,AFRICA,0.221757,700,155.229844
3,2021-10-18 07:20:00+00:00,ASIA,0.336539,1100,370.193310
4,2021-10-18 07:30:00+00:00,AMERICAS,0.149375,1000,149.374868


In [22]:
risk = beam_df.copy()

def normalize_series(s):
    s = s.astype(float)
    return ((s - s.min()) / (s.max() - s.min() + 1e-9)).fillna(0.0)

# ---------- Jamming / interference risk ----------

jamming_risk = pd.Series(0.0, index=idx)

jamming_path = BASE_DIR / "ses_comm_features.csv"
if jamming_path.exists():
    try:
        comm = pd.read_csv(jamming_path, parse_dates=["time"])
        comm = comm.set_index("time").sort_index()
        if "anomaly_score" in comm.columns:
            jam = comm["anomaly_score"].copy()
            jam_10 = jam.groupby(jam.index.floor("10min")).mean()
            jam_10 = normalize_series(jam_10)
            jamming_risk = jam_10.reindex(idx).interpolate().fillna(0.0)
            print("Loaded jamming risk from ses_comm_features.csv")
        else:
            print("ses_comm_features.csv has no anomaly_score column, using synthetic jamming risk.")
    except ValueError as e:
        print(f"Error parsing date column 'time' from {jamming_path}: {e}. Using synthetic jamming risk.")
else:
    print("ses_comm_features.csv not found, using synthetic jamming risk.")

if jamming_risk.max() == 0:
    # Synthetic: a few random high-risk nights
    base = rng.uniform(0, 0.2, size=len(idx))
    spikes_idx = rng.choice(len(idx), size=int(0.02 * len(idx)), replace=False)
    base[spikes_idx] += rng.uniform(0.4, 0.8, size=len(spikes_idx))
    jamming_risk = pd.Series(np.clip(base, 0, 1), index=idx)

# Broadcast to beams
risk["jamming_risk"] = jamming_risk.loc[risk["time"]].values

# ---------- SLA breach risk ----------

sla_flags_path = BASE_DIR / "artifacts_sla" / "sla_flags_test.csv"
sla_risk = pd.Series(0.0, index=idx)

if sla_flags_path.exists():
    try:
        sla = pd.read_csv(sla_flags_path, parse_dates=["_time"])
        sla = sla.set_index("_time").sort_index()
        if "BREACH_ANY" in sla.columns:
            s = sla["BREACH_ANY"].astype(float)
            s_10 = s.groupby(s.index.floor("10min")).max()
            # smooth via rolling mean
            s_10 = s_10.rolling(6, min_periods=1).mean()
            sla_risk = normalize_series(s_10).reindex(idx).fillna(0.0)
            print("Loaded SLA risk from artifacts_sla/sla_flags_test.csv")
        else:
            print("BREACH_ANY not found in sla_flags_test, using synthetic SLA risk.")
    except ValueError as e:
        print(f"Error parsing date column 'time' from {sla_flags_path}: {e}. Using synthetic SLA risk.")
else:
    print("SLA flags file not found, using synthetic SLA risk.")

if sla_risk.max() == 0:
    base = rng.uniform(0, 0.15, size=len(idx))
    spikes_idx = rng.choice(len(idx), size=int(0.01 * len(idx)), replace=False)
    base[spikes_idx] += rng.uniform(0.5, 0.9, size=len(spikes_idx))
    sla_risk = pd.Series(np.clip(base, 0, 1), index=idx)

risk["sla_risk"] = sla_risk.loc[risk["time"]].values

# ---------- Signal loss risk (if file exists, otherwise synthetic) ----------

sig_path = BASE_DIR / "artifacts_signal_loss" / "test_scores.csv"
sig_risk = pd.Series(0.0, index=idx)

if sig_path.exists():
    try:
        sig = pd.read_csv(sig_path, parse_dates=["timestamp"]) #timestamp
        sig = sig.set_index("timestamp").sort_index()
        # Try to find a probability / score column
        prob_col = None
        for c in sig.columns:
            if "proba" in c.lower() or "score" in c.lower():
                prob_col = c
                break
        if prob_col:
            s = sig[prob_col].astype(float)
            s_10 = s.groupby(s.index.floor("10min")).max()
            sig_risk = normalize_series(s_10).reindex(idx).fillna(0.0)
            print("Loaded signal-loss risk from artifacts_signal_loss/test_scores.csv")
        else:
            print("No probability column in signal-loss test_scores.csv, using synthetic risk.")
    except ValueError as e:
        print(f"Error parsing date column 'time' from {sig_path}: {e}. Using synthetic signal-loss risk.")
else:
    print("Signal-loss test_scores.csv not found, using synthetic risk.")

if sig_risk.max() == 0:
    base = rng.uniform(0, 0.1, size=len(idx))
    spikes_idx = rng.choice(len(idx), size=int(0.005 * len(idx)), replace=False)
    base[spikes_idx] += rng.uniform(0.6, 1.0, size=len(spikes_idx))
    sig_risk = pd.Series(np.clip(base, 0, 1), index=idx)

risk["signal_loss_risk"] = sig_risk.loc[risk["time"]].values

# ---------- Beam-handover risk ----------

handover_path = BASE_DIR / "artifacts_handover" / "test_scores.csv"
handover_risk = pd.Series(0.0, index=idx)

if handover_path.exists():
    try:
        ho = pd.read_csv(handover_path, parse_dates=["timestamp"]) #timestamp
        ho = ho.set_index("timestamp").sort_index()
        prob_col = None
        for c in ho.columns:
            if "proba" in c.lower() or "score" in c.lower():
                prob_col = c
                break
        if prob_col:
            s = ho[prob_col].astype(float)
            s_10 = s.groupby(s.index.floor("10min")).max()
            handover_risk = normalize_series(s_10).reindex(idx).fillna(0.0)
            print("Loaded handover risk from artifacts_handover/test_scores.csv")
        else:
            print("No probability column in handover test_scores.csv, using synthetic risk.")
    except ValueError as e:
        print(f"Error parsing date column 't' from {handover_path}: {e}. Using synthetic handover risk.")
else:
    print("Handover test_scores.csv not found, using synthetic risk.")

if handover_risk.max() == 0:
    base = rng.uniform(0, 0.05, size=len(idx))
    spikes_idx = rng.choice(len(idx), size=int(0.01 * len(idx)), replace=False)
    base[spikes_idx] += rng.uniform(0.3, 0.7, size=len(spikes_idx))
    handover_risk = pd.Series(np.clip(base, 0, 1), index=idx)

risk["handover_risk"] = handover_risk.loc[risk["time"]].values

Loaded jamming risk from ses_comm_features.csv
Loaded SLA risk from artifacts_sla/sla_flags_test.csv
Loaded signal-loss risk from artifacts_signal_loss/test_scores.csv
Loaded handover risk from artifacts_handover/test_scores.csv


In [25]:
space_path = BASE_DIR / "ses_spaceweather_dataset.csv"
space_risk = pd.Series(0.0, index=idx)

if space_path.exists():
    sw = pd.read_csv(space_path, parse_dates=["time"])
    sw = sw.sort_values("time")
    y = sw["maneuver_risky"].astype(int)
    feature_cols = [
        "duration_sec", "is_unload", "is_stationkeeping",
        "thr_temp_mean", "thr_temp_max", "thr_temp_min",
        "thr_temp_std", "thr_temp_slope",
        "att_err_mean", "att_err_max", "att_err_std",
        "Kp", "ap", "Kp_prev12h_max", "Kp_prev24h_max",
        "is_storm", "storm_intensity",
    ]
    X = sw[feature_cols].fillna(0)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    clf_sw = HistGradientBoostingClassifier(random_state=42)
    clf_sw.fit(X_train, y_train)
    y_prob = clf_sw.predict_proba(X_test)[:, 1]
    print("Space-weather risky maneuver ROC-AUC:",
          roc_auc_score(y_test, y_prob))

    # Predict probabilities for all maneuvers
    sw["risk_prob"] = clf_sw.predict_proba(X)[:, 1]

    # Project to global 10-min grid by nearest time
    sw = sw.set_index("time").sort_index()
    sw_prob = sw["risk_prob"].reindex(idx, method="nearest", tolerance=pd.Timedelta("3h"))
    sw_prob = sw_prob.fillna(0.0)
    space_risk = normalize_series(sw_prob)
    print("Space-weather risk projected to global time grid.")
else:
    print("ses_spaceweather_dataset.csv not found, using synthetic space-weather risk.")

if space_risk.max() == 0:
    base = rng.uniform(0, 0.05, size=len(idx))
    spikes_idx = rng.choice(len(idx), size=int(0.01 * len(idx)), replace=False)
    base[spikes_idx] += rng.uniform(0.4, 0.9, size=len(spikes_idx))
    space_risk = pd.Series(np.clip(base, 0, 1), index=idx)

risk["space_weather_risk"] = risk["time"].map(space_risk)

risk.head()

Space-weather risky maneuver ROC-AUC: 0.48698830409356725
Space-weather risk projected to global time grid.


,time,beam_id,demand_norm,capacity_mbps,demand_mbps,jamming_risk,sla_risk,signal_loss_risk,handover_risk,space_weather_risk
0,2021-10-18 07:20:00+00:00,AMERICAS,0.278428,1000,278.427797,0.298021,0.0,0.0,0.0,0.034222
1,2021-10-18 07:20:00+00:00,EUROPE,0.184874,900,166.386189,0.298021,0.0,0.0,0.0,0.034222
2,2021-10-18 07:20:00+00:00,AFRICA,0.221757,700,155.229844,0.298021,0.0,0.0,0.0,0.034222
3,2021-10-18 07:20:00+00:00,ASIA,0.336539,1100,370.193310,0.298021,0.0,0.0,0.0,0.034222
4,2021-10-18 07:30:00+00:00,AMERICAS,0.149375,1000,149.374868,0.185325,0.0,0.0,0.0,0.029479


In [26]:
# Merge risk channels into beam_df
for col in ["jamming_risk", "sla_risk", "signal_loss_risk",
            "handover_risk", "space_weather_risk"]:
    beam_df[col] = risk[col]

# Normalise demand separately for the model
beam_df["demand_risk"] = normalize_series(beam_df["demand_norm"])

# Weights for the composite risk index (tuneable!)
WEIGHTS = {
    "demand_risk":        0.40,
    "sla_risk":           0.20,
    "signal_loss_risk":   0.15,
    "jamming_risk":       0.10,
    "handover_risk":      0.05,
    "space_weather_risk": 0.10,
}

beam_df["risk_index"] = 0.0
for k, w in WEIGHTS.items():
    beam_df["risk_index"] += w * beam_df[k]

beam_df["risk_index"] = beam_df["risk_index"].clip(0, 1.0)

beam_df[["time", "beam_id", "demand_norm", "risk_index"]].head()


,time,beam_id,demand_norm,risk_index
0,2021-10-18 07:20:00+00:00,AMERICAS,0.278428,0.124470
1,2021-10-18 07:20:00+00:00,EUROPE,0.184874,0.087100
2,2021-10-18 07:20:00+00:00,AFRICA,0.221757,0.101833
3,2021-10-18 07:20:00+00:00,ASIA,0.336539,0.147682
4,2021-10-18 07:30:00+00:00,AMERICAS,0.149375,0.061176


In [27]:
base_prob = 0.02 + 0.75 * beam_df["risk_index"]
incident_prob = base_prob.clip(0, 0.95)

rng = np.random.default_rng(123)
beam_df["capacity_incident"] = rng.binomial(1, incident_prob)

beam_df["capacity_incident"].mean()


np.float64(0.1565082644628099)

In [28]:
feature_cols = [
    "demand_risk",
    "sla_risk",
    "signal_loss_risk",
    "jamming_risk",
    "handover_risk",
    "space_weather_risk",
]

X = beam_df[feature_cols].values
y = beam_df["capacity_incident"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

clf = HistGradientBoostingClassifier(
    learning_rate=0.1, max_depth=5, max_iter=200, random_state=42
)
clf.fit(X_train, y_train)

y_hat = clf.predict_proba(X_test)[:, 1]
print("Capacity incident ROC-AUC:", roc_auc_score(y_test, y_hat))
print(classification_report(y_test, (y_hat > 0.5).astype(int)))


Capacity incident ROC-AUC: 0.7093111387220594
              precision    recall  f1-score   support

           0       0.84      1.00      0.91      5307
           1       0.39      0.01      0.03       985

    accuracy                           0.84      6292
   macro avg       0.62      0.51      0.47      6292
weighted avg       0.77      0.84      0.78      6292



In [29]:
explainer = shap.TreeExplainer(clf, X_train, feature_names=feature_cols)
shap_values = explainer(X, check_additivity=False)

# Global mean |SHAP|
mean_abs_shap = np.abs(shap_values.values).mean(axis=0)
shap_global = pd.DataFrame({
    "feature": feature_cols,
    "mean_abs_shap": mean_abs_shap
}).sort_values("mean_abs_shap", ascending=False)

shap_global

100%|===================| 25151/25168 [00:30<00:00]       

,feature,mean_abs_shap
0,demand_risk,0.550907
2,signal_loss_risk,0.199489
1,sla_risk,0.123211
3,jamming_risk,0.055144
4,handover_risk,0.053740
5,space_weather_risk,0.048104


In [30]:
plt.figure(figsize=(8, 5))
shap.plots.bar(shap_values, show=False)
plt.tight_layout()
plt.savefig(CAP_DIR / "capacity_risk_shap_bar.png", dpi=200)
plt.close()


In [31]:
ts_out = beam_df[[
    "time", "beam_id",
    "demand_mbps", "capacity_mbps", "demand_norm",
    "jamming_risk", "sla_risk", "signal_loss_risk",
    "handover_risk", "space_weather_risk",
    "risk_index", "capacity_incident"
]].copy()

ts_out.to_csv(CAP_DIR / "capacity_risk_timeseries.csv", index=False)
shap_global.to_csv(CAP_DIR / "capacity_risk_shap_global.csv", index=False)

print("Saved:")
print(" -", CAP_DIR / "capacity_risk_timeseries.csv")
print(" -", CAP_DIR / "capacity_risk_shap_global.csv")


Saved:
 - /content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/artifacts_capacity/capacity_risk_timeseries.csv
 - /content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/artifacts_capacity/capacity_risk_shap_global.csv


In [32]:
capacity_risk_timeseries.csv
capacity_risk_shap_global.csv


NameError: name 'capacity_risk_timeseries' is not defined

In [33]:
# =====================================================
# Extra step: heuristic "Resilience-Aware Load Balancing"
# =====================================================

print("Building heuristic capacity re-balancing suggestions...")

ts = ts_out.copy()

# 1. Aggregate per beam per hour (you can change to 30min / 1 day if you prefer)
ts_hour = (
    ts.set_index("time")
      .groupby("beam_id")
      .resample("1H")
      .agg({
          "demand_mbps": "mean",
          "capacity_mbps": "mean",
          "risk_index": "mean",
      })
      .reset_index()
      .dropna()
)

recs = []
for t, df_t in ts_hour.groupby("time"):
    if len(df_t) < 2:
        continue

    # highest- and lowest-risk beams in that hour
    df_sorted = df_t.sort_values("risk_index", ascending=False)
    high = df_sorted.iloc[0]
    low = df_sorted.iloc[-1]

    # how much spare capacity does the low-risk beam have?
    slack = max(low["capacity_mbps"] - low["demand_mbps"], 0.0)

    # simple heuristic: move up to 10% of high-beam demand,
    # but not more than 50% of the low-beam slack
    move_mbps = min(high["demand_mbps"] * 0.10, slack * 0.50)

    if move_mbps < 1.0:  # tiny moves are not interesting
        continue

    # toy estimate of risk reduction
    est_reduction = (
        (high["risk_index"] - low["risk_index"])
        * (move_mbps / (high["demand_mbps"] + 1e-6))
    )

    recs.append({
        "time_start": t,
        "beam_from": high["beam_id"],
        "beam_to": low["beam_id"],
        "mbps_move": round(move_mbps, 1),
        "risk_from": round(float(high["risk_index"]), 3),
        "risk_to": round(float(low["risk_index"]), 3),
        "est_risk_reduction": round(float(est_reduction), 4),
    })

if recs:
    recs_df = (
        pd.DataFrame(recs)
        .sort_values("est_risk_reduction", ascending=False)
        .reset_index(drop=True)
    )
    recs_path = CAP_DIR / "capacity_recommendations.csv"
    recs_df.to_csv(recs_path, index=False)
    print(f"Saved {len(recs_df)} recommendations to {recs_path}")
else:
    print("No non-trivial re-balancing suggestions found; "
          "capacity_recommendations.csv not created.")

Building heuristic capacity re-balancing suggestions...


/tmp/ipython-input-2379889906.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample("1H")


Saved 1049 recommendations to /content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/artifacts_capacity/capacity_recommendations.csv
